# Part 1: Computer Vision Functions
***
# Table of Contents
1.   [Setup](#Setup)
2.   [Exercise 1 - Connected Components](#Exercise-1---Connected-Components)
3.   [Exercise 2 - Dilation](#Exercise-2---Dilation)
4.   [Exercise 3 - Erosion](#Exercise-3---Erosion)
5.   [Exercise 4 - Opening](#Exercise-4---Opening)
6.   [Exercise 5 - Closing](#Exercise-5---Closing)
7.   [Exercise 6 - Line Segmentation](#Exercise-6---Line-Segmentation)


# Stage 1 Setup

In [1]:
import cv2
import numpy as np

# Object Extraction

Given a scene containing objects and a mask of a desired object, the following function will extract the object from the image.

In [2]:
def ExtractObject(S2, ObjectMask):
    return cv2.bitwise_and(S2, S2, mask=ObjectMask)
    

S2 = cv2.imread("Images/Books/3_colour.jpeg",1)

ObjectMask = cv2.imread("Masks/Books/lpbooks_no_3_colour_mask_2_mask.png")

ExtractedObject = ExtractObject(S2, ObjectMask)
cv2.imwrite("Output/Extracted Object.png", ExtractedObject, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Filtering

* 5x5 averaging
* Gaussian Blurring
* Median Blurring

In [3]:
def ApplyFilter(ExtractedObject, FilterIndex):
    if FilterIndex == 0:
        return ExtractedObject
    
    if FilterIndex == 1:
        # 2D Convolution 5x5 averaging
        kernel = np.ones((5,5),np.float32)/25
        return cv2.filter2D(ExtractedObject, -1, kernel)
    
    if FilterIndex == 2:
        # Gaussian Blurring
        return cv2.GaussianBlur(ExtractedObject, (5,5), 0)
    
    if FilterIndex == 3:
        # Median Blurring
        return cv2.medianBlur(ExtractedObject, 5)

cv2.imwrite("Output/Filtered Object 0.png", ApplyFilter(ExtractedObject.copy(), 0), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 1.png", ApplyFilter(ExtractedObject.copy(), 1), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 2.png", ApplyFilter(ExtractedObject.copy(), 2), [cv2.IMWRITE_PNG_COMPRESSION, 0])
cv2.imwrite("Output/Filtered Object 3.png", ApplyFilter(ExtractedObject.copy(), 3), [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Object Blending

In [4]:
def ObjectBlending(S1, FilteredExObject):
    
    c_text = S1.copy()
    for i, y in enumerate(FilteredExObject):
        for j, x in enumerate(y):
            if np.sum(FilteredExObject[i][j]) != 0:
                c_text[i][j] = x 
    return c_text

S1 = cv2.imread("Images/Books/1_colour.jpeg")
BlendingResult = ObjectBlending(S1, ExtractedObject)
cv2.imwrite("Output/Blended Result.png", BlendingResult, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

# Stage 2

# Remove Green

Generate hsv for documentation
https://stackoverflow.com/questions/47483951/how-to-define-a-threshold-value-to-detect-only-green-colour-objects-in-an-image

In [37]:
def removeGreen(img):
    # convert to hsv
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)

    # mask of green (36,25,25) ~ (86, 255,255)
    mask = cv2.inRange(hsv, (36, 25, 25), (86, 255,255))

    # slice the green
    imask = mask > 0
    green = np.zeros_like(img, np.uint8)
    green[imask] = img[imask]
    cv2.imwrite("Output/only Green.png", green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip to gray
    gray = cv2.cvtColor(green.copy(),cv2.COLOR_BGR2GRAY)
    cv2.imwrite("Output/gray.png", gray, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # get treshold
    ret, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY)
    cv2.imwrite("Output/tresh.png", thresh, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    # flip
    flip = cv2.bitwise_not(thresh)
    cv2.imwrite("Output/flip.png", flip, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    kernel = np.ones((20, 20), 'uint8')
    close = cv2.morphologyEx(flip, cv2.MORPH_CLOSE ,kernel)
    cv2.imwrite("Output/close.png", close, [cv2.IMWRITE_PNG_COMPRESSION, 0])

    c_img = img.copy()
    for i, y in enumerate(img):
        for j, x in enumerate(y):
            if np.sum(close[i][j]) != 255:
                c_img[i][j] = (0, 0, 0)
    return c_img

img = cv2.imread("Images/Books/1_colour.jpeg")
no_green = removeGreen(img.copy())
cv2.imwrite("Output/No Green.png", no_green, [cv2.IMWRITE_PNG_COMPRESSION, 0])

True

In [ ]:
def ObjectBlending(S1, FilteredExObject):
    
    c_text = S1.copy()
    for i, y in enumerate(FilteredExObject):
        for j, x in enumerate(y):
            if np.sum(FilteredExObject[i][j]) != 0:
                c_text[i][j] = x 
    return c_text

S1 = cv2.imread("Images/Books/1_colour.jpeg",1)
BlendingResult = ObjectBlending(S1, ExtractedObject)
cv2.imshow("Blended Scene", BlendingResult)
cv2.waitKey(0)